##### 版权所有 2019 TensorFlow 作者。

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 自定义联合算法，第 1 部分：联合核心简介

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_1"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.62.0/docs/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

本教程是由两部分组成的系列的第一部分，该系列演示如何使用[Federated Core (FC)](../federated_core.md)在 TensorFlow Federated (TFF) 中实现自定义类型的联合算法 - 一组较低级别的接口，用作基础我们已经实现了[联邦学习（FL）](../federated_learning.md)层。

第一部分更具概念性；我们介绍了 TFF 中使用的一些关键概念和编程抽象，并在一个带有分布式温度传感器阵列的非常简单的示例中演示了它们的使用。在[本系列的第二部分](custom_federated_algorithms_2.ipynb)中，我们使用此处介绍的机制来实现联邦训练和评估算法的简单版本。作为后续，我们鼓励您研究`tff.learning`中联邦平均的[实现](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/python/learning/algorithms/fed_avg.py)。

读完本系列后，您应该能够认识到 Federated Core 的应用不一定仅限于学习。我们提供的编程抽象非常通用，可以用来实现分布式数据的分析和其他自定义类型的计算。

尽管本教程被设计为独立的，但我们鼓励您首先阅读有关[图像分类](federated_learning_for_image_classification.ipynb)和[文本生成](federated_learning_for_text_generation.ipynb)的教程，以更深入、更温和地介绍 TensorFlow 联合框架和[联合学习](../federated_learning.md)API ( `tff.learning` )，如它将帮助您将我们在此描述的概念放在上下文中。

## 预期用途 1

简而言之，Federated Core (FC) 是一个开发环境，可以紧凑地表达将 TensorFlow 代码与分布式通信运算符相结合的程序逻辑，例如[联邦平均](https://arxiv.org/abs/1602.05629)中使用的运算符 - 计算分布式总和、平均值和其他类型系统中一组客户端设备上的分布式聚合，向这些设备广播模型和参数等。

您可能知道[`tf.contrib.distribute`](https://www.tensorflow.org/api_docs/python/tf/contrib/distribute) ，此时自然要问的问题可能是：这个框架有何不同？毕竟，这两个框架都试图使 TensorFlow 计算变得分布式。

一种思考方式是，虽然`tf.contrib.distribute`的既定目标是*允许用户使用现有模型和训练代码，只需进行最少的更改即可实现分布式训练*，而重点是如何利用分布式基础设施为了使现有的训练代码更加高效，TFF 联邦核心的目标是让研究人员和从业者能够明确控制他们将在系统中使用的分布式通信的特定模式。 FC 的重点是提供一种灵活且可扩展的语言来表达分布式数据流算法，而不是一组具体的已实现的分布式训练功能。

TFF FC API 的主要目标受众之一是研究人员和从业者，他们可能想要尝试新的联邦学习算法并评估影响分布式系统中数据流编排方式的微妙设计选择的后果，但不会被系统实现细节所困扰。 FC API 所追求的抽象级别大致对应于研究出版物中用于描述联邦学习算法机制的伪代码 - 系统中存在哪些数据以及如何转换数据，但不会下降到以下级别：单独的点对点网络消息交换。

TFF 作为一个整体的目标是数据分布的场景，并且必须保持这种分布，例如出于隐私原因，并且在集中位置收集所有数据可能不是一个可行的选择。与所有数据都可以累积在数据中心的集中位置的场景相比，这对需要增加显式控制程度的机器学习算法的实现产生了影响。

## 在我们开始之前

在我们深入研究代码之前，请尝试运行以下“Hello World”示例，以确保您的环境设置正确。如果不起作用，请参阅[安装](../install.md)指南以获取说明。

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## 联合数据

TFF 的显着特征之一是它允许您在*联合数据*上紧凑地表达基于 TensorFlow 的计算。我们将在本教程中使用术语*联合数据*来指代分布式系统中一组设备上托管的数据项的集合。例如，在移动设备上运行的应用程序可以收集数据并将其存储在本地，而不上传到集中位置。或者，分布式传感器阵列可以收集并存储其所在位置的温度读数。

像上面示例中的联合数据在 TFF 中被视为[一等公民](https://en.wikipedia.org/wiki/First-class_citizen)，即它们可能显示为函数的参数和结果，并且它们具有类型。为了强化这一概念，我们将联合数据集称为*联合值*或*联合类型的值*。

需要理解的重要一点是，我们将所有设备上的整个数据项集合（例如，分布式阵列中所有传感器的整个集合温度读数）建模为单个联合值。

例如，以下是如何在 TFF 中定义由一组客户端设备托管的*联合浮动*类型。通过分布式传感器阵列实现的温度读数集合可以建模为这种联合类型的值。

In [ ]:
federated_float_on_clients = tff.type_at_clients(tf.float32)

更一般地，TFF 中的联合类型是通过指定其*成员组成部分*的类型`T`来定义的 - 驻留在各个设备上的数据项，以及托管此类型的联合值的设备组`G` （加上第三个、我们很快就会提到的可选信息）。我们将托管联合值的设备组`G`称为值的*放置*。因此， `tff.CLIENTS`是一个放置示例。

In [ ]:
str(federated_float_on_clients.member)

'float32'

In [ ]:
str(federated_float_on_clients.placement)

'CLIENTS'

具有成员成分`T`和位置`G`联合类型可以紧凑地表示为`{T}@G` ，如下所示。

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

这种简洁表示法中的大括号`{}`旨在提醒您，成员组成部分（不同设备上的数据项）可能会有所不同，如您所期望的，例如温度传感器读数，因此客户端作为一个组共同托管[多个](https://en.wikipedia.org/wiki/Multiset)-一起构成联合值的`T`类型项目的[集合](https://en.wikipedia.org/wiki/Multiset)。

值得注意的是，联合值的成员组成部分通常对程序员来说是不透明的，即联合值不应被视为由系统中设备的标识符键入的简单`dict` - 这些值旨在只能由抽象代表各种分布式通信协议（例如聚合）的*联邦算子*来集体转换。如果这听起来太抽象，请不要担心 - 我们很快就会回到这个话题，并用具体的例子来说明它。

TFF 中的联合类型有两种类型：联合值的成员成分可能不同的类型（如上所示），以及已知它们全部相等的类型。这是由`tff.FederatedType`构造函数中的第三个可选`all_equal`参数控制的（默认为`False` ）。

In [ ]:
federated_float_on_clients.all_equal

False

具有放置`G`的联合类型，其中已知所有`T`类型成员成分都相等，可以紧凑地表示为`T@G` （与`{T}@G`相对，即删除大括号以反映事实上，多组成员成分由单个项目组成）。

In [ ]:
str(tff.type_at_clients(tf.float32, all_equal=True))

'float32@CLIENTS'

在实际场景中可能出现的此类联合值的一个示例是已由服务器广播到参与联合训练的一组设备的超参数（例如学习率、裁剪范数等）。

另一个例子是在服务器上预先训练的机器学习模型的一组参数，然后将其广播到一组客户端设备，在那里可以针对每个用户进行个性化设置。

例如，假设我们有一对用于简单一维线性回归模型的`float32`参数`a`和`b` 。我们可以构建此类模型的（非联合）类型以在 TFF 中使用，如下所示。打印类型字符串中的尖括号`<>`是命名或未命名元组的紧凑 TFF 表示法。

In [ ]:
simple_regression_model_type = (
    tff.StructType([('a', tf.float32), ('b', tf.float32)]))

str(simple_regression_model_type)

'<a=float32,b=float32>'

请注意，我们在上面仅指定了`dtype` 。还支持非标量类型。在上面的代码中， `tf.float32`是更通用的`tff.TensorType(dtype=tf.float32, shape=[])`的快捷表示法。

当该模型被广播到客户端时，所得到的联合值的类型可以如下所示表示。

In [ ]:
str(tff.type_at_clients(
    simple_regression_model_type, all_equal=True))

'<a=float32,b=float32>@CLIENTS'

根据上面*联合浮点数的*对称性，我们将这种类型称为*联合元组*。更一般地，我们经常使用术语*“联合 XYZ”*来指代其中成员成分类似于*XYZ*的联合值。因此，我们将讨论诸如*联合元组*、*联合序列*、*联合模型*等内容。

现在，回到`float32@CLIENTS` - 虽然它看起来是跨多个设备复制的，但它实际上是单个`float32` ，因为所有成员都是相同的。一般来说，您可能会认为任何*全等联合*类型（即`T@G`形式之一）与非联合类型`T`同构，因为在这两种情况下，实际上只有一个（尽管可能会复制）项类型`T` 。

鉴于`T`和`T@G`之间的同构，您可能想知道后一种类型可能有什么用途（如果有的话）。请继续阅读。

## 展示位置

### 设计概述

在上一节中，我们介绍了*放置*的概念 - 可能共同托管联合值的系统参与者组，并且我们演示了如何使用`tff.CLIENTS`作为放置的示例规范。

为了解释为什么*布局*的概念如此重要以至于我们需要将其合并到 TFF 类型系统中，请回想一下我们在本教程开头提到的有关 TFF 的一些预期用途的内容。

尽管在本教程中，您只会看到 TFF 代码在模拟环境中本地执行，但我们的目标是让 TFF 能够编写可以部署在分布式系统中的物理设备组（可能包括移动或嵌入式设备）上执行的代码运行安卓。这些设备中的每一个都将接收一组单独的指令以在本地执行，具体取决于它在系统中扮演的角色（最终用户设备、集中协调器、多层架构中的中间层等）。能够推断哪些设备子集执行哪些代码以及数据的不同部分可能在哪里物理实现非常重要。

当处理例如移动设备上的应用程序数据时，这一点尤其重要。由于数据是私有的并且可能是敏感的，因此我们需要能够静态验证该数据永远不会离开设备（并证明有关数据处理方式的事实）。放置规范是旨在支持这一点的机制之一。

TFF 被设计为以数据为中心的编程环境，因此，与一些专注于*操作*以及这些操作可能*运行的*位置的现有框架不同，TFF 专注于*数据*、数据在何处*具体化*以及如何*转换*。因此，放置被建模为 TFF 中数据的属性，而不是数据操作的属性。事实上，正如您将在下一节中看到的，一些 TFF 操作跨越多个位置，并且可以说“在网络中”运行，而不是由单个计算机或一组计算机执行。

将某个值的类型表示为`T@G`或`{T}@G` （而不只是`T` ）使得数据放置决策变得明确，并且与用 TFF 编写的程序的静态分析一起，它可以作为提供敏感设备数据的正式隐私保证。

然而，此时需要注意的重要一点是，虽然我们鼓励 TFF 用户明确托管数据（放置）的参与设备*组*，但程序员永远不会处理*单个*参与者的原始数据或身份。

在 TFF 代码体内，根据设计，无法枚举构成`tff.CLIENTS`表示的组的设备，也无法探测组中是否存在特定设备。在 Federated Core API、底层架构抽象集或我们为支持模拟而提供的核心运行时基础设施中，任何地方都没有设备或客户端身份的概念。你编写的所有计算逻辑都会表现为对整个客户端组的操作。

回想一下我们之前提到的联合类型的值与 Python `dict`不同的地方，因为联合类型的值不能简单地枚举它们的成员成分。将 TFF 程序逻辑操作的值视为与展示位置（组）相关联，而不是与单个参与者相关联。

展示位置*也*被设计为 TFF 中的一等公民，并且可以显示为`placement`类型的参数和结果（由 API 中的`tff.PlacementType`表示）。将来，我们计划提供各种运算符来转换或组合展示位置，但这超出了本教程的范围。现在，只需将`placement`视为 TFF 中的不透明基元内置类型就足够了，类似于 Python 中`int`和`bool`是不透明内置类型， `tff.CLIENTS`是这种类型的常量文字，与`1`不同是`int`类型的常量文字。

### 指定展示位置

TFF 提供了两个基本的放置文字`tff.CLIENTS`和`tff.SERVER` ，以便轻松表达丰富多样的实际场景，这些场景自然地建模为客户端-服务器架构，具有多个*客户端*设备（手机、嵌入式设备、分布式数据库） 、传感器等）由单个集中式*服务器*协调器编排。 TFF 还设计为支持自定义布局、多个客户端组、多层和其他更通用的分布式架构，但讨论它们超出了本教程的范围。

TFF 没有规定`tff.CLIENTS`或`tff.SERVER`实际代表什么。

特别是， `tff.SERVER`可能是单个物理设备（单例组的成员），但它也可能是运行状态机复制的容错集群中的一组副本 - 我们没有做任何特殊的架构假设。相反，我们使用上一节中提到的`all_equal`位来表达这样一个事实：我们通常只处理服务器上的单个数据项。

同样，某些应用程序中的`tff.CLIENTS`可能代表系统中的所有客户端 - 在联邦学习的背景下，我们有时将其称为*总体*，但例如，在[联邦平均的生产实现](https://arxiv.org/abs/1602.05629)中，它可能代表一个*群组*- 的子集选择参加特定一轮培训的客户。当部署它们出现的计算来执行（或者像模拟环境中的 Python 函数一样简单地调用，如本教程中所示）时，抽象定义的放置将被赋予具体含义。在我们的本地模拟中，客户端组由作为输入提供的联合数据确定。

## 联合计算

### 声明联合计算

TFF 被设计为支持模块化开发的强类型函数式编程环境。

TFF 中的基本组成单元是*联合计算*- 可以接受联合值作为输入并返回联合值作为输出的逻辑部分。以下是您如何定义一个计算，以计算上一个示例中传感器阵列报告的温度平均值。

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):
  return tff.federated_mean(sensor_readings)

看看上面的代码，此时您可能会问 - TensorFlow 中是否已经存在用于定义可组合单元（例如[`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function)的装饰器构造，如果是，为什么还要引入另一个装饰器构造，它有何不同？

简而言之， `tff.federated_computation`包装器生成的代码*既不*是 TensorFlow，*也不是*Python - 它是采用内部平台无关的*胶水*语言的分布式系统规范。在这一点上，这无疑听起来很神秘，但请牢记联合计算作为分布式系统的抽象规范的直观解释。我们将在一分钟内解释它。

首先，让我们稍微了解一下这个定义。 TFF 计算通常被建模为函数 - 带或不带参数，但具有明确定义的类型签名。您可以通过查询其`type_signature`属性来打印计算的类型签名，如下所示。

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

类型签名告诉我们，计算接受客户端设备上不同传感器读数的集合，并在服务器上返回单个平均值。

在我们进一步讨论之前，让我们思考一下这一点 - 该计算的输入和输出*位于不同的位置*（在`CLIENTS`端上与在`SERVER`上）。回想一下我们在上一节关于*TFF 操作如何跨位置并在网络中运行的布局中所说的内容*，以及我们刚才所说的关于代表分布式系统抽象规范的联合计算的内容。我们刚刚定义了一个这样的计算 - 一个简单的分布式系统，其中数据在客户端设备上使用，聚合结果出现在服务器上。

在许多实际场景中，代表顶级任务的计算往往会在服务器上接受其输入并报告其输出 - 这反映了计算可能由在服务器上发起和终止的*查询*触发的想法。

然而，FC API 并没有强加这种假设，并且我们内部使用的许多构建块（包括您可能在 API 中找到的许多`tff.federated_...`运算符）具有具有不同位置的输入和输出，因此一般来说，您应该不要将联合计算视为*在服务器上运行*或*由服务器执行的计算*。服务器只是联合计算中的一种参与者。在考虑此类计算的机制时，最好始终默认为全球网络范围的视角，而不是单个集中协调器的视角。

一般来说，对于输入和输出的类型`T`和`U` ，函数类型签名分别紧凑地表示为`(T -> U)` 。形式参数的类型（例如本例中的`sensor_readings` ）被指定为装饰器的参数。您不需要指定结果的类型 - 它是自动确定的。

尽管 TFF 确实提供了有限形式的多态性，但强烈鼓励程序员明确他们所使用的数据类型，因为这使得理解、调试和形式验证代码的属性变得更加容易。在某些情况下，需要显式指定类型（例如，多态计算当前不能直接执行）。

### 执行联合计算

为了支持开发和调试，TFF 允许您直接调用以这种方式定义的计算作为 Python 函数，如下所示。如果计算需要`all_equal`位设置为`False`的联合类型的值，您可以将其作为 Python 中的普通`list`提供，而对于`all_equal`位设置为`True`联合类型，您可以直接提供（单个）成员组成。这也是向您报告结果的方式。

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

在模拟模式下运行这样的计算时，您充当具有系统范围视图的外部观察者，他能够在网络中的任何位置提供输入并使用输出，就像这里的情况一样 - 您提供了客户端值在输入时，并消耗服务器结果。

现在，让我们回到之前关于`tff.federated_computation`装饰器以*胶水*语言发出代码的注释。尽管 TFF 计算的逻辑可以用 Python 中的普通函数表示（您只需像我们上面所做的那样用`tff.federated_computation`修饰它们），并且您可以像本文中的任何其他 Python 函数一样直接使用 Python 参数调用它们Notebook，在幕后，正如我们之前提到的，TFF 计算实际上*不是*Python。

我们的意思是，当 Python 解释器遇到用`tff.federated_computation`修饰的函数时，它会跟踪该函数主体中的语句一次（在定义时），然后构造计算逻辑的[序列化表示](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/proto/v0/computation.proto)以供将来使用 - 无论是用于执行，或作为子组件合并到另一个计算中。

您可以通过添加打印语句来验证这一点，如下所示：

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):

  print ('Getting traced, the argument is "{}".'.format(
      type(sensor_readings).__name__))

  return tff.federated_mean(sensor_readings)

Getting traced, the argument is "Value".


您可以将定义联合计算的 Python 代码想象成类似于在非热切上下文中构建 TensorFlow 图的 Python 代码（如果您不熟悉 TensorFlow 的非热切用法，请考虑您的Python 代码定义了稍后要执行的操作图，但实际上并没有实时运行它们）。 TensorFlow 中的非热切图构建代码是 Python，但该代码构建的 TensorFlow 图是平台无关且可序列化的。

同样，TFF 计算是在 Python 中定义的，但其主体中的 Python 语句（例如我们刚刚显示的示例中的`tff.federated_mean` ）在底层被编译为可移植且与平台无关的可序列化表示形式。

作为开发人员，您不需要关心这种表示的细节，因为您永远不需要直接使用它，但您应该意识到它的存在，事实上 TFF 计算基本上是非渴望的，并且无法捕获任意Python状态。 TFF 计算主体中包含的 Python 代码在定义时执行，此时在序列化之前跟踪用`tff.federated_computation`修饰的 Python 函数主体。它不会在调用时再次回溯（除非该函数是多态的；有关详细信息，请参阅文档页面）。

您可能想知道为什么我们选择引入专用的内部非 Python 表示形式。原因之一是，TFF 计算最终旨在可部署到真实的物理环境中，并托管在移动或嵌入式设备上，而 Python 可能无法在这些设备上使用。

另一个原因是 TFF 计算表达了分布式系统的全局行为，而不是表达单个参与者的本地行为的 Python 程序。您可以在上面的简单示例中看到，使用特殊运算符`tff.federated_mean`接受客户端设备上的数据，但将结果存储在服务器上。

运算符`tff.federated_mean`无法轻松地建模为 Python 中的普通运算符，因为它不在本地执行 - 如前所述，它代表协调多个系统参与者行为的分布式系统。我们将此类运算符称为*联合运算符*，以将它们与 Python 中的普通（本地）运算符区分开来。

因此，TFF 类型系统以及 TFF 语言支持的基本操作集与 Python 中的显着不同，因此需要使用专用表示。

### 组合联合计算

如上所述，联合计算及其组成部分最好理解为分布式系统的模型，并且您可以将组合联合计算视为从更简单的分布式系统组合更复杂的分布式系统。您可以将`tff.federated_mean`运算符视为一种具有类型签名`({T}@CLIENTS -> T@SERVER)`的内置模板联合计算（事实上，就像您编写的计算一样，该运算符也有一个复杂的结构 - 在底层我们将其分解为更简单的运算符）。

组合联合计算也是如此。可以在用`tff.federated_computation`装饰的另一个 Python 函数的主体中调用计算`get_average_temperature` - 这样做将导致它嵌入到父函数的主体中，这与`tff.federated_mean`之前嵌入到其自己的主体中的方式非常相似。

需要注意的一个重要限制是，用`tff.federated_computation`修饰的 Python 函数体必须*仅*包含联合运算符，即它们不能直接包含 TensorFlow 运算。例如，您不能直接使用`tf.nest`接口来添加一对联合值。 TensorFlow 代码必须仅限于使用下一节中讨论的`tff.tf_computation`修饰的代码块。只有以这种方式包装时，才能在`tff.federated_computation`的主体中调用包装的 TensorFlow 代码。

这种分离的原因是技术性的（很难欺骗`tf.add`等运算符来处理非张量）和架构性的。联合计算的语言（即，从用`tff.federated_computation`修饰的 Python 函数的序列化主体构建的逻辑）被设计为充当独立于平台的*粘合*语言。这种胶水语言目前用于从 TensorFlow 代码的嵌入式部分（仅限于`tff.tf_computation`块）构建分布式系统。随着时间的推移，我们预计需要嵌入其他非 TensorFlow 逻辑的部分，例如可能代表输入管道的关系数据库查询，所有这些都使用相同的粘合语言（ `tff.federated_computation`块）连接在一起。

## TensorFlow逻辑

### 声明 TensorFlow 计算

TFF 设计用于与 TensorFlow 结合使用。因此，您将在 TFF 中编写的大部分代码可能是普通（即本地执行）TensorFlow 代码。为了将此类代码与 TFF 一起使用，如上所述，只需用`tff.tf_computation`进行修饰。

例如，下面是我们如何实现一个接受数字并加上`0.5`函数。

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

再次看到这一点，您可能想知道为什么我们应该定义另一个装饰器`tff.tf_computation`而不是简单地使用现有的机制，例如`tf.function` 。与上一节不同的是，这里我们处理的是普通的 TensorFlow 代码块。

造成这种情况的原因有几个，其完整处理超出了本教程的范围，但值得指出主要的一个：

- 为了将使用 TensorFlow 代码实现的可重用构建块嵌入到联合计算主体中，它们需要满足某些属性 - 例如在定义时进行跟踪和序列化、具有类型签名等。这通常需要某种形式的装饰器。

一般来说，我们建议尽可能使用 TensorFlow 的本机组合机制，例如`tf.function` ，因为 TFF 装饰器与 eager 函数交互的确切方式预计会不断发展。

现在，回到上面的示例代码片段，我们刚刚定义的计算`add_half`可以像任何其他 TFF 计算一样由 TFF 处理。特别是，它具有 TFF 类型签名。

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

请注意，此类型签名没有展示位置。 TensorFlow 计算无法使用或返回联合类型。

您现在还可以使用`add_half`作为其他计算中的构建块。例如，以下是如何使用`tff.federated_map`运算符将`add_half`逐点应用于客户端设备上联合浮动的所有成员组成部分。

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

### 执行 TensorFlow 计算

使用`tff.tf_computation`定义的计算的执行遵循与我们为`tff.federated_computation`描述的规则相同的规则。它们可以作为 Python 中的普通可调用对象进行调用，如下所示。

In [ ]:
add_half_on_clients([1.0, 3.0, 2.0])

[<tf.Tensor: shape=(), dtype=float32, numpy=1.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.5>]

再次值得注意的是，以这种方式调用计算`add_half_on_clients`模拟了分布式过程。数据在客户端上消费，并在客户端上返回。事实上，这种计算让每个客户端执行本地操作。该系统中没有明确提及`tff.SERVER` （即使在实践中，编排此类处理可能涉及一个）。以这种方式定义的计算在概念上类似于`MapReduce`中的`Map`阶段。

另外，请记住，我们在上一节中所说的关于在定义时间序列化 TFF 计算的内容对于`tff.tf_computation`代码也同样适用 - `add_half_on_clients`的 Python 主体在定义时间被跟踪一次。在后续调用中，TFF 使用其序列化表示。

用`tff.federated_computation`修饰的 Python 方法与用`tff.tf_computation`修饰的 Python 方法之间的唯一区别是后者被序列化为 TensorFlow 图（而前者不允许包含直接嵌入其中的 TensorFlow 代码）。

在底层，用`tff.tf_computation`修饰的每个方法都会暂时禁用急切执行，以便允许捕获计算的结构。虽然本地禁用了 eager 执行，但欢迎您使用 eager TensorFlow、AutoGraph、TensorFlow 2.0 构造等，只要您以能够正确序列化的方式编写计算逻辑即可。

例如，以下代码将失败：

In [ ]:
try:

  # Eager mode
  constant_10 = tf.constant(10.)

  @tff.tf_computation(tf.float32)
  def add_ten(x):
    return x + constant_10

except Exception as err:
  print (err)

Attempting to capture an EagerTensor without building a function.


上面的方法失败了，因为在序列化过程中， `constant_10`已经在`tff.tf_computation`在`add_ten`主体内部构建的图表之外构建了。

另一方面，在`tff.tf_computation`内部调用时调用修改当前图的 python 函数是可以的：

In [ ]:
def get_constant_10():
  return tf.constant(10.)

@tff.tf_computation(tf.float32)
def add_ten(x):
  return x + get_constant_10()

add_ten(5.0)

15.0

请注意，TensorFlow 中的序列化机制正在不断发展，我们预计 TFF 如何序列化计算的细节也会不断发展。

### 使用`tf.data.Dataset`

如前所述， `tff.tf_computation`的一个独特功能是它们允许您使用由代码抽象定义为形式参数的`tf.data.Dataset` 。要在 TensorFlow 中表示为数据集的参数需要使用`tff.SequenceType`构造函数进行声明。

例如，类型规范`tff.SequenceType(tf.float32)`定义了 TFF 中 float 元素的抽象序列。序列可以包含张量或复杂的嵌套结构（稍后我们将看到这些示例）。 `T`类型项目序列的简洁表示是`T*` 。

In [ ]:
float32_sequence = tff.SequenceType(tf.float32)

str(float32_sequence)

'float32*'

假设在我们的温度传感器示例中，每个传感器不仅保存一个温度读数，而且保存多个温度读数。以下是如何在 TensorFlow 中定义 TFF 计算，该计算使用`tf.data.Dataset.reduce`运算符计算单个本地数据集中的平均温度。

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def get_local_temperature_average(local_temperatures):
  sum_and_count = (
      local_temperatures.reduce((0.0, 0), lambda x, y: (x[0] + y, x[1] + 1)))
  return sum_and_count[0] / tf.cast(sum_and_count[1], tf.float32)

In [ ]:
str(get_local_temperature_average.type_signature)

'(float32* -> float32)'

在用`tff.tf_computation`修饰的方法主体中，TFF 序列类型的形式参数简单地表示为行为类似于`tf.data.Dataset`的对象，即支持相同的属性和方法（它们当前未实现为 tf.data.Dataset 的子类）该类型 - 随着 TensorFlow 中数据集支持的发展，这可能会发生变化）。

您可以按如下方式轻松验证这一点。

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.int32))
def foo(x):
  return x.reduce(np.int32(0), lambda x, y: x + y)

foo([1, 2, 3])

6

请记住，与普通的`tf.data.Dataset`不同，这些类似数据集的对象是占位符。它们不包含任何元素，因为它们表示抽象序列类型参数，在具体上下文中使用时将绑定到具体数据。目前对抽象定义的占位符数据集的支持仍然受到一定限制，并且在 TFF 的早期，您可能会遇到某些限制，但在本教程中我们不需要担心它们（请参阅文档页面）了解详情）。

当本地执行在模拟模式下接受序列的计算时（例如在本教程中），您可以将序列作为 Python 列表提供，如下所示（以及以其他方式，例如，作为 eager 中的`tf.data.Dataset`模式，但现在我们将保持简单）。

In [ ]:
get_local_temperature_average([68.5, 70.3, 69.8])

69.53333

与所有其他 TFF 类型一样，上面定义的序列可以使用`tff.StructType`构造函数来定义嵌套结构。例如，以下是如何声明一个计算，该计算接受一系列`A` 、 `B` ，并返回它们的乘积之和。我们将跟踪语句包含在计算主体中，以便您可以看到 TFF 类型签名如何转换为数据集的`output_types`和`output_shapes` 。

In [ ]:
@tff.tf_computation(tff.SequenceType(collections.OrderedDict([('A', tf.int32), ('B', tf.int32)])))
def foo(ds):
  print('element_structure = {}'.format(ds.element_spec))
  return ds.reduce(np.int32(0), lambda total, x: total + x['A'] * x['B'])

element_structure = OrderedDict([('A', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('B', TensorSpec(shape=(), dtype=tf.int32, name=None))])


In [ ]:
str(foo.type_signature)

'(<A=int32,B=int32>* -> int32)'

In [ ]:
foo([{'A': 2, 'B': 3}, {'A': 4, 'B': 5}])

26

尽管在简单场景（例如本教程中使用的场景）中起作用，但对使用`tf.data.Datasets`作为形式参数的支持仍然受到一定限制并不断发展。

## 把它们放在一起

现在，让我们再次尝试在联合设置中使用 TensorFlow 计算。假设我们有一组传感器，每个传感器都有一个本地温度读数序列。我们可以通过对传感器的局部平均值进行平均来计算全局温度平均值，如下所示。

In [ ]:
@tff.federated_computation(
    tff.type_at_clients(tff.SequenceType(tf.float32)))
def get_global_temperature_average(sensor_readings):
  return tff.federated_mean(
      tff.federated_map(get_local_temperature_average, sensor_readings))

请注意，这并不是所有客户端的所有本地温度读数的简单平均值，因为这需要根据不同客户端本地维护的读数数量来权衡不同客户端的贡献。我们将其作为练习留给读者更新上述代码； `tff.federated_mean`运算符接受权重作为可选的第二个参数（预计是联合浮点数）。

另请注意， `get_global_temperature_average`的输入现在变成了*联合浮点序列*。联合序列是我们在联合学习中通常表示设备上数据的方式，序列元素通常表示数据批次（您很快就会看到这样的示例）。

In [ ]:
str(get_global_temperature_average.type_signature)

'({float32*}@CLIENTS -> float32@SERVER)'

以下是我们如何在 Python 中本地执行数据样本的计算。请注意，我们提供输入的方式现在是`list`的`list` 。外部列表迭代`tff.CLIENTS`表示的组中的设备，内部列表迭代每个设备本地序列中的元素。

In [ ]:
get_global_temperature_average([[68.0, 70.0], [71.0], [68.0, 72.0, 70.0]])

70.0

本教程的第一部分到此结束...我们鼓励您继续学习[第二部分](custom_federated_algorithms_2.ipynb)。